In [5]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(3, 384)
tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [15]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# Load Hugging Face embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_5iAUBD_QTr7hP54SA6DGtuhgGynT35UwcjykiGKdCEqxyYcGCU6rEbg6m2AUPeHuq3H6AC")  # Replace with your actual API key
index_name = "text-search"

# Connect to existing Pinecone index
if index_name not in pc.list_indexes().names():
    raise ValueError(f"Index '{index_name}' does not exist. Please create it in the Pinecone console.")

index = pc.Index(index_name)

def embed_text(text):
    """Generate embeddings."""
    return model.encode(text).tolist()

def store_text(text, id):
    """Store embedded text into Pinecone."""
    embedding = embed_text(text)
    vectors = [(id, embedding, {"text": text})]
    index.upsert(vectors=vectors)
    print(f"Stored: {vectors}")

def search_text(query, top_k=1):
    """Perform a similarity search in Pinecone."""
    query_embedding = embed_text(query)
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    if results and results["matches"]:
        best_match = results["matches"][0]
        return best_match["metadata"]["text"], best_match["score"]
    return None, None

if __name__ == "__main__":
    # Store sample texts (only needed once)
    sample_data = [
        ("1", "Passport Number: X12345678, Name: John Doe, Date of Birth: 15th July 1990, Nationality: United States, Expiry Date: 12th Dec 2030"),
        ("2", "Passport Number: Y98765432, Name: Alice Smith, Date of Birth: 22nd March 1985, Nationality: Canada, Expiry Date: 8th Aug 2028"),
        ("3", "Passport Number: Z55511122, Name: Robert Johnson, Date of Birth: 5th May 1975, Nationality: United Kingdom, Expiry Date: 20th Nov 2025")
    ]
    
    for id, text in sample_data:
        store_text(text, id)

    # Clearly prompt the user
    print("\nPlease enter your question below:")
    query = input("> ").strip()  # Takes user input

    # Perform search and print results
    result_text, score = search_text(query)

    if result_text:
        print(f"\nBest match: {result_text}\nScore: {score:.4f}")
    else:
        print("\nNo match found.")


Stored: [('1', [-0.09529300034046173, 0.03894798457622528, 0.027164625003933907, -0.014163845218718052, 0.02300218679010868, 0.062445588409900665, -0.03544861078262329, -0.025611788034439087, -0.04916348680853844, 0.02453685738146305, -0.0009860050631687045, -0.017175229266285896, -0.0019352077506482601, -0.0249358918517828, -0.07296556234359741, -0.016500189900398254, -0.07276914268732071, -0.036960501223802567, 0.04119539260864258, -0.035417430102825165, 0.0333489254117012, 0.05407135561108589, -0.036968402564525604, -0.12115941941738129, 0.006394304800778627, 0.013991705141961575, 0.009721457026898861, 0.021395018324255943, -0.018322192132472992, 0.027724288403987885, 0.03778686001896858, 0.05089329555630684, -0.029157890006899834, 0.05587282031774521, 0.05788026750087738, -0.00794418528676033, -0.032623883336782455, 0.01662549190223217, -0.01658523827791214, -0.0165344700217247, -0.024047156795859337, -0.09193256497383118, 0.034083619713783264, 0.06540337949991226, 0.03558186069130